Kredi Kartları Skor Verisi İle Makine Öğrenmesi

Kaggle sitesindeki https://www.kaggle.com/datasets/ayushsharma0812/dataset-for-credit-score-classification
linkindeki Kresi Skor verisetinin (Dataset for Credit Score Classification) keşifsel veri analizi aşağıda sunulmuştur.

Öncelikle panda kütüphanesi kullanılarak csv formatındaki veri kümesi okunur.

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/kredi-skor-verisi/credit_score.csv')

Sayısal Keşifsel Veri Analizi

In [ ]:
df.info()

df.info komutu veri kümesi ile ilgili genel bilgiler verir. Veri kümesinde 28 değişken vardır. Bu veri kümesinde 100000 kayıt vardır. Veri kümesinden Credit_Score değişkeni y olarak seçilir. Diğer değişkenlerden x seçilerek model olusturulur. Kolonların içerdiği değere göre alınan non-null countlarda farklı değerler olması verilerin null değerler içerdiğini göstermektedir.

df.describe() metodu ile veri alanları ile ilgili özet bilgilerle veri açıklanmaya ve analiz için fayda sağlayacak bilgiler edinilmeye çalışılacaktır.

In [ ]:
df.describe(include="all")

Bu analiz sonucunda credit_score sahasının 3 farklı değer içermesi, sınıflandırmada tahmin edilecek y değişkeni olacağını göstermektedir. Analizden veri setinin Occupation ve Credit-Mix  isimli kategorik değişkenler içerdiği görülmektedir. 

In [ ]:
df["Credit_Score"].describe()

df.head() komutu ile verilerin ilk 5 tanesi listelenir. Listelenen verileri inceleyerek verilerle ilgili gerekl, ön işleme süreeci belirlenecektir.

Veri setinde Nan değerler olması veri seti üzerinde nan değerleri düzeltecek bir veri temizleme sürecinin gerekli olduğunu göstermektedir.

In [ ]:
df.head()

Veri kümesinde eksik verilerin olduğu açık olarak görülmektedir. Şimdi ise eksik verilerin sayılarını belirleyelim. 

In [ ]:
print(df.isnull().sum().sort_values(ascending=False))

Monthly_Inhand_Salary, Type_of_Loan, Name, Credit_History_Age, Num_of_Delayed_Payment, Amount_invested_monthly Num_Credit_Inquiries, Monthly_Balance sahalarında boş veriler olduğu yukarıda gözükmektedir.

Boş Kayıtlar

Profile report üzerinden verisetinde değeri null olan kolonlar görülmektedir.  Monthly_Inhand_Salary, Type_of_Loan, Name, Credit_History_Age, Num_of_Delayed_Payment,Amount_invested_monthly,Num_Credit_Inquiries,Monthly_Balance sahalarında null değerler olduğu profile report ve yukarıdaki null değerler sorgulamada gözükmektedir.

Bu durumda boş değerli kayıtlar silinerek veya istatistiki bir yöntemle doldurularak işletilen bir veri temizleme yönteminden sonra yola devam edilir.

Değişkenler
Veri setindeki 28 değişkenin, 13'ün metin, 6'sının kategorik, 8'inin nümerik olduğu profil raporundan görükmektedir. Bu analiz modele katılacak değişkenleri belirlemek açısından önemlidir. Nümerik ve kategorik değişkenler üzerine odaklanmak sağlıklı görülmektedir.

Korelasyon
Profile reportta verisetinde birbirleri arasında yüksek korelasyon olan kayıtlar olduğu görülmektedir. 
Delay_from_due_date ile Interest_Rate arasında ve Num_bank_accounts ile Delay_from_due_date arasında yüksek korelasyon olduğu görülmektedir. Yüksek korelasyonlu değişkenleri belirlemek model açısından değişken azaltmada önemlidir. Yüksek korelasyon olan değişkenlerin birlikte modelde olması, biri varken diğerinin modele katkısını azaltmakta, hatta karmaşıklığı artırdığı için sorun teşkil etmektedir. 

In [ ]:
!pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df)
profile.to_notebook_iframe()

Kategorik Değişkenler

Aşağıdaki df.nunique() metodu çıktısında 6 kategorik değişken olduğu görülmektedir.
Bu değişkenler,  Month, Occupation, Credit_Mix, Payment_of_Min_Amount, Payment_Behaviour, Credit_Score dur.

In [ ]:
df.nunique()

In [ ]:
dfModel = df.copy()


In [ ]:
dfModel.head(10)

In [ ]:
dfModel.columns

Kurulması planlanan modelde anlamı olmayacak sahalar model dataframe'inden aşağıdaki komutla silinir.

In [ ]:
dfModel = dfModel.drop(['Month','ID', 'Customer_ID',  'Name',  'SSN','Type_of_Loan','Credit_History_Age'], axis=1)


In [ ]:
dfModel.columns

In [ ]:
dfModel.head(10)

Veri Temizleme

Sahalarda anlam ifade etmeyen veriler bulunan veri satırları silinerek modelden atılır.

In [ ]:
dfModel["Payment_Behaviour"].unique()
dfModel["Payment_Behaviour"].count

In [ ]:
len(dfModel["Payment_Behaviour"].unique())

Payment_Behaviour değişkeni değerindeki !@9#%8 anlamsız değeri bulunan satır aşağıdaki komutla veri setinden silinmiştir. 

In [ ]:
dfModel = dfModel.loc[df['Payment_Behaviour'] != '!@9#%8'] 

In [ ]:
dfModel["Payment_Behaviour"].unique()

!@9#%8 verisini içeren satırın silindiği yukarıda görülmektedir.

Bu işlem sonucunda veri sayısının 92400'e düştüğü aşağıda görülmektedir.

In [ ]:
dfModel["Payment_Behaviour"].count

Boş Kayıtların Silinmesi

dropna() komutu ile veri setindeki nan değer içeren sahaların bulunduğu satırlar veri setinden silinir.

In [ ]:
drop_dataframe = dfModel.dropna()



Aşağıda çalıştırılan drop_dataframe.isna() komutu verisetinde nan değer kalmadığını göstermektedir.
Bu işlem sonucunda veri kümesindeki kayıt sayısı 67676' ya düşmüştür.

In [ ]:
drop_dataframe.isna()

In [ ]:
drop_dataframe.head(10)

Veri setinde Occupation ve Credit_Mix kategoriK değişkenlerinde aşağıda belirtilen anlamsız değerler belirlenmiştir.
Occupation sahası _______ değeri olan satırları modelden silelim.
Credit_Mix  sahası _______ değeri olan satırları modelden silelim.
Bu anlamsız değerleri içeren satırlar silindekten sonra kategorik değişkenleri etiketlemeye geçeçöeceğiz.


In [ ]:
drop_dataframe = drop_dataframe.loc[df['Occupation'] != '_______'] 
drop_dataframe["Occupation"].unique()

drop_dataframe = drop_dataframe.loc[df['Credit_Mix'] != '_'] 
drop_dataframe["Credit_Mix"].unique()

Bu temizlenme faaliyetlerinden sonra kalan kayıtlar aşağıdaki metod ile listelenir.

In [ ]:
drop_dataframe.head(10)

Bu işlemden sonra elimizde 50144 kayıt kaldığı aşağıda görülmektedir.

In [ ]:
drop_dataframe["Credit_Mix"]

In [ ]:
drop_dataframe

Model verileri kullanılarak öğrenme admına geçmeden önceki son veri temizleme adımı modeldeki kategorik değişkenleri label encodin algoritması ile nümerik değerler ile etiketlemek olacaktır.

Modeldeki kategorik değişkenlerimiz, Occupation, Credit_Mix,Payment_of_Min_Amount,Payment_Behaviour,Credit_Score

In [ ]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows  
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
drop_dataframe['COccupation']= label_encoder.fit_transform(drop_dataframe['Occupation']) 
  
drop_dataframe['COccupation'].unique() 
drop_dataframe[['COccupation',"Occupation"]]
#Credit_Mix,Payment_of_Min_Amount,Payment_Behaviour,Credit_Score
drop_dataframe['CCredit_Mix']= label_encoder.fit_transform(drop_dataframe['Credit_Mix']) 
drop_dataframe['CPayment_of_Min_Amount']= label_encoder.fit_transform(drop_dataframe['Payment_of_Min_Amount']) 
drop_dataframe['CPayment_Behaviour']= label_encoder.fit_transform(drop_dataframe['Payment_Behaviour'])
drop_dataframe['CCredit_Score']= label_encoder.fit_transform(drop_dataframe['Credit_Score'])


drop_dataframe[['COccupation',"Occupation", "CCredit_Mix","Credit_Mix","CPayment_of_Min_Amount","Payment_of_Min_Amount", "CPayment_Behaviour","Payment_Behaviour","CCredit_Score","Credit_Score"]]

Yukarıdaki tablodan görüldüğü üzere kategorik değişkenler 0 dan başlayan sayı değerleri ile kodlanmıştır.

In [ ]:
drop_dataframe.head(10)

Kategorik variable kendi isimlerine set edip yeni kolonları silelim.

In [ ]:
drop_dataframe.drop(['Occupation','Credit_Mix','Payment_of_Min_Amount','Payment_Behaviour','Credit_Score'],inplace=True, axis=1)

drop_dataframe.rename(columns={'COccupation': 'Occupation', 'CCredit_Mix': 'Credit_Mix', 'CPayment_of_Min_Amount': 'Payment_of_Min_Amount','CPayment_Behaviour': 'Payment_Behaviour','CCredit_Score': 'Credit_Score'}, inplace=True)
drop_dataframe.info()

In [ ]:
drop_dataframe.head(10)

Nümerik Değer İçermeyen sahaları temizleyelim.

In [ ]:
numeric_df = drop_dataframe.apply(pd.to_numeric, errors='coerce')
numeric_df =numeric_df.dropna()
numeric_df.info()


Veri kümesi kolonlarında nümerik olmayan değer içeren satırlar yukarıdaki komut ile temizlendikten sonra elimizde 37846 kayıt kalır.

Univariate Analysis

In [ ]:
# Load libraries
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt 

# Create the plot
sns.boxplot(x='Annual_Income', data=numeric_df)
plt.show()

In [ ]:
# Create the plot
sns.boxplot(y='Annual_Income', x="Credit_Score", hue="Credit_Score",  data=numeric_df,palette="Set3", linewidth=1)
plt.show()

Aşağıdaki kod parçası ile m modelin x ve y veri kümesi oluşturulur.

In [ ]:
#dfModel     = drop_dataframe
y_DataSet = numeric_df["Credit_Score"]
x_DataSet = numeric_df.copy()
x_DataSet = x_DataSet.drop(["Credit_Score"], axis=1)

Model 19 değişkenden oluşur. Veri kümesindeki 19 değişken kullanılarak oluşturulan model Credit_Score y değişkenini tahmin etmeye çalışır.

In [ ]:
x_DataSet.info()
x_DataSet.head(10)

In [ ]:
y_DataSet.head(10)

Gözetimli Öğrenme Algoritmaları:

Decision Tree Algoritması

In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, RepeatedKFold,cross_val_score



clf = DecisionTreeClassifier(random_state=42)

sk_folds = StratifiedKFold(n_splits = 5)

scores = cross_val_score(clf, x_DataSet, y_DataSet, cv = sk_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

scores = cross_val_score(clf, x_DataSet, y_DataSet, cv = rkf)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))



Cross validation score'lara yaklaşık %67 doğrulukta tahmin yapılmıştır. Şimdi KNN algoritmasını deneyelim.

KNN Algoritması

In [ ]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, RepeatedKFold,cross_val_score



clf = KNeighborsClassifier(n_neighbors = 5)

sk_folds = StratifiedKFold(n_splits = 5)

scores = cross_val_score(clf, x_DataSet, y_DataSet, cv = sk_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

scores = cross_val_score(clf, x_DataSet, y_DataSet, cv = rkf)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))



KNN Algoritmasında Cross validation score'lara yaklaşık %62 doğrulukta tahmin yapılmıştır.

Gözetimsiz Algoritmalar.

In [ ]:
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold, RepeatedKFold,cross_val_score



clf =  KMeans(n_clusters=8)

sk_folds = StratifiedKFold(n_splits = 5)

scores = cross_val_score(clf, x_DataSet, y_DataSet, cv = sk_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

scores = cross_val_score(clf, x_DataSet, y_DataSet, cv = rkf)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))


Gaussian Mixture Algoritmaları

Aşağıda aussian Mixture Algoritmasu ile sınıflandırna ve olasılık hesaplama işlemleri gözükmektedir.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

# create GM ModelsN
model = GaussianMixture(n_components=3,
                        random_state=42).fit(x_DataSet)

gmm_model = model.predict(x_DataSet)
xk = numeric_df.to_numpy()
plt.scatter(xk[:, 0], xk[:, 1], c=gmm_model, cmap='viridis')


probs = model.predict_proba(x_DataSet)
prob_table =pd.DataFrame(data=probs)
prob_table.head()


Hiperparametre Tuning

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x_DataSet,y_DataSet,test_size=0.2,random_state=1)
#parm grid is dictionry for tuning hyperperameters of Decision Tree
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [3, 7, 12],
    'min_samples_leaf': [2, 8, 10]
}
dtc = DecisionTreeClassifier(random_state=42)
#Performing GridSearch to find the best hyperparameters
grid_search = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)

#Training the decision tree model with the best hyperparameters
best_dt_model = grid_search.best_estimator_

#Evaluating the model on the testing set
y_pred = best_dt_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
grid_search.best_score_

Decision Tree sınıflandırıcısında hiperparemetre tuning sonucunda başarım %67 den %70'e çıktı.
Birde değişkenlere normalizasyon ekleyelim.



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Initialize the scaler
scaler = StandardScaler()


x_train,x_test,y_train,y_test= train_test_split(x_DataSet,y_DataSet,test_size=0.2,random_state=1)
#parm grid is dictionry for tuning hyperperameters of Decision Tree

# Fit and transform the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [3, 7, 12],
    'min_samples_leaf': [2, 8, 10]
}
dtc = DecisionTreeClassifier(random_state=42)
#Performing GridSearch to find the best hyperparameters
grid_search = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)

#Training the decision tree model with the best hyperparameters
best_dt_model = grid_search.best_estimator_

#Evaluating the model on the testing set
y_pred = best_dt_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
grid_search.best_score_

x Değişkenlerinin normalize edilmesi sonuca etki etmedi.


Şimdi Knn için Hiperparametre tuning yapalım.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Initialize the scaler
scaler = StandardScaler()


x_train,x_test,y_train,y_test= train_test_split(x_DataSet,y_DataSet,test_size=0.2,random_state=1)
#parm grid is dictionry for tuning hyperperameters of Decision Tree

# Fit and transform the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

param_grid = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
dtc = KNeighborsClassifier()
#Performing GridSearch to find the best hyperparameters
grid_search = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)

#Training the decision tree model with the best hyperparameters
best_dt_model = grid_search.best_estimator_

#Evaluating the model on the testing set
y_pred = best_dt_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
grid_search.best_score_

KNN Algoritmasında hiper parametre tuning sonucunda  yaklaşık %66 doğrulukta tahmin ile öncekine göre %4 lük iyileştirme sağlanmıştır.